## Query LSSTCam

In this notebook, we show how to query LSSTCam\
Craig Lage - 15-Apr-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.plotting import plot
from lsst.obs.lsst import LsstCam

In [ ]:
# This doesn't work, because it has LSSTCam/quickLook instead of LSSTCam/runs/quickLook
butler = butlerUtils.makeDefaultButler("LSSTCam")
coll = butler.collections
for item in coll:
    print(item)

In [ ]:
# This works
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", "LSSTCam/runs/quickLook"])

In [ ]:
expId = 2025102200009
raw = butler.get('raw', detector=94, exposure=expId)
mData = raw.getMetadata()
for key in mData.keys():
    print(key, mData[key])

In [ ]:
expId = 2025102100110
calexp = butler.get('preliminary_visit_image', visit=expId, detector=94)

In [ ]:
expId = 2025102200009
camera = LsstCam().getCamera()
detectors = [det.getId() for det in camera]
for detector in detectors:
    try:
        raw = butler.get('raw', exposure=expId, detector=detector)
        #calexp = butler.get('preliminary_visit_image', visit=expId, detector=detector)
        mData = raw.getMetadata()
        print(detector, mData['ROTCOORD'])
    except:
        print(f"Detector {detector} failed!")
        continue

In [ ]:
import random, time
tests = [[20251021, [94, 126]], [20250918, [100, 200]], [20250919, [100, 200]], \
 [20250920, [100, 200]], [20250921, [100, 200]]]
times = []
for i in range(1000):
    rand1 = 0#random.randint(0,4)
    dayObs = tests[rand1][0]
    seqNum = random.randint(tests[rand1][1][0], tests[rand1][1][0])
    expId = int(dayObs * 1E5 + seqNum)
    detector = random.randint(0, 188)
    #print(dayObs, seqNum, expId, detector)
    t1 = time.time()
    raw = butler.get('raw', exposure=expId, detector=detector)
    t2 = time.time()
    delay = t2 - t1
    times.append(delay)


In [ ]:
plt.title("Random butler.get response times in summit nublado")
plt.hist(times, bins=50, range=(0,2)) 
plt.xlabel("Response time (seconds)")
plt.savefig(f"/home/cslage/DATA/Butler_Response_22Oct25.png")

In [ ]:
raw = butler.get('raw', exposure=expId, detector=110)
mData = raw.getMetadata()
print(mData['FILTER'])

In [ ]:
collections = butler.registry.queryCollections()
for item in collections:
    print(item)

In [ ]:
registry = butler.registry
for dtype in registry.queryDatasetTypes()[:]:
    try:
        datasetRefs = list(registry.queryDatasets(datasetType=dtype,collections=butler.registry.queryCollections()))
        if len(datasetRefs) > 0:
            print(len(datasetRefs), dtype )
    except:
        print("Error", dtype )

In [ ]:
expId = 2025042500591
calexp = butler.get('raw', detector=94, exposure=expId)

In [ ]:
expId = 2025042500591
calexp = butler.get('calexp', detector=94, visit=expId)

In [ ]:
x = plot(postISRCCD, )

In [ ]:
dataId={'instrument':'LSSTCam', 'detector':90, 'exposure':expId}

In [ ]:
uri = butler.getURI('raw', dataId=dataId)
uri

In [ ]:
butler.getURI?

In [ ]:
from lsst.summit.utils.efdUtils import makeEfdClient
from astropy.time import Time, TimeDelta
client = makeEfdClient()

In [ ]:
start = Time("2025-10-21T21:30:00", scale='utc')
end = Time("2025-10-21T22:15:00", scale='utc')
msg = await client.select_time_series("lsst.sal.MTHeaderService.logevent_logMessage", \
                                            ['message'], \
                                             start, end)
len(msg)

In [ ]:
for message in msg['message']:
    if "Timeout while waiting for MTCamera_0_endOfImageTelemetry Event" in message:
        print(message)

In [ ]:
start = Time("2025-10-21T12:30:00", scale='utc')
end = Time("2025-10-21T22:15:00", scale='utc')
msg = await client.select_time_series("lsst.sal.MTCamera.logevent_endOfImageTelemetry", \
                                            ['imageName'], \
                                             start, end)
len(msg)

In [ ]:
for message in msg['imageName']:
    print(message)